In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
!pip install tokenizers
!pip install transformers
!pip install sentencepiece
!pip install beautifulsoup4
!pip install selectolax

     |████████████████████████████████| 6.6 MB 8.5 MB/s 
     |████████████████████████████████| 4.0 MB 9.2 MB/s 
     |████████████████████████████████| 895 kB 51.4 MB/s 
     |████████████████████████████████| 77 kB 5.9 MB/s 
     |████████████████████████████████| 596 kB 33.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 8.3 MB/s 
     |████████████████████████████████| 4.9 MB 8.4 MB/s 


In [3]:
import re
import csv
import html
import nltk 
import json
import random

from nltk import tokenize
from typing import List, Dict, Any
from bs4 import BeautifulSoup
from selectolax.parser import HTMLParser

In [4]:
class CsvIterator:
  def __init__(self, csv_file_path, row_names: List[str], return_column_name):
    self.f = open(csv_file_path)
    self.csv_reader = csv.reader(self.f)
    self.row_names = row_names
    self.return_column_name = return_column_name
    self.return_index = self.row_names.index(self.return_column_name)
    self.sentences = []
    self.i = 0

  def _fill_sentences(self):
    row = self.csv_reader.__next__()
    text = html.unescape(row[self.return_index])
    # text = replace_html_tags(text, ['a', 'img'])
    while not self.sentences:
      self.sentences += [sent for sent in text.split('\n') if len(sent) < 100 and sent != '']

  def __iter__(self):
    return self

  def __next__(self):
    row =  self.csv_reader.__next__()
    if self.i <= 1_000_000:
      self.i += 1
      # BS = BeautifulSoup(html.unescape(row[self.return_index]))
      # print(row[self.return_index])
      # if '<a' in row[self.return_index]:
      #   print(html.unescape(row[self.return_index]))
      #   print('-' * 50)
      #   print(replace_html_tags(replace_html_tags(html.unescape(row[self.return_index]), 'a'), 'img'))
      #   print('#' * 50, end='\n\n')
      # length, ret = 0, ''
      # if BS.code:
      #   for code in BS.code.contents:
      #     if len(code) > length:
      #       length = len(code)
      #       ret = str(code)
      if not self.sentences:
        self._fill_sentences()
      return self.sentences.pop()
      # return replace_html_tags(html.unescape(row[self.return_index]), ['a', 'img'])
      # print(match)
      # if match:
      #   return match.groups()[0]
    else:
      raise StopIteration

  def __del__(self):
    self.f.close()


def replace_html_tags(string, tags):
  soup = BeautifulSoup(string)
  for tag in tags:
    for value in soup.findAll(tag):
      value.replaceWithChildren()
  return str(soup)

def process_html(body: str):
    tree = HTMLParser(body)
    tree.unwrap_tags(['img', 'a'])
    text = html.unescape(tree.body.html)
    return [sent for sent in text.split('\n') if sent != '']

def read_csv(path: str, row_names: List[str]) -> List[Dict[str, Any]]:
  data = []
  with open(path) as f:
    reader = csv.reader(f)
    for row in reader:
      entry = {}
      for name, value in zip(row_names, row):
        entry[name] = value
      data.append(entry)
  return data

In [5]:
question_rows = ['id', 'title', 'tags', 'body', 'acceptedAnswerId', 'score', 'views']
questions = read_csv('drive/Shared drives/685 Final Project/Stackoverflow Data/large_questions.csv', question_rows)

In [6]:
questions = [question['body'] for question in questions]

In [7]:
question_sentences = []
for question in questions:
  sents = process_html(question)
  question_sentences += sents if len(sents) <= 3 else random.sample(sents, 3)
questions = None

In [8]:
from tokenizers import (SentencePieceUnigramTokenizer, SentencePieceBPETokenizer, Tokenizer)
from tokenizers.pre_tokenizers import Whitespace
from transformers import DebertaV2Tokenizer

daberta_tokenizer = DebertaV2Tokenizer.from_pretrained('microsoft/deberta-v3-small')

Downloading:   0%|          | 0.00/2.35M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/578 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
def make_new_vocabulary(tokenizer, existing_tokenizer, train_data, path, **train_kwargs):
  vocab = existing_tokenizer.get_vocab()
  tokenizer.train_from_iterator(train_data, **train_kwargs)
  new_vocab = tokenizer.get_vocab()

  diff_vocab = {k:v for k,v in new_vocab.items() if k not in vocab}

  print(f'diff size {len(diff_vocab)}')

  offset = existing_tokenizer.vocab_size
  new_tokens = []
  for i, (token, id) in enumerate(diff_vocab.items()):
    new_tokens.append(token)

  existing_tokenizer.add_tokens(new_tokens)
  existing_tokenizer.save_pretrained(path)

In [11]:
len(question_sentences)

4691148

In [19]:
question_sentences[7]

'Scaling dynos... done, now running worker at 1:Free'

In [20]:
tokenizer = SentencePieceBPETokenizer()
# question_sentences = question_sentences, 5_000_000)
save_path = 'drive/Shared drives/685 Final Project/Code Search Pre-Trained Model/tokenizer'
make_new_vocabulary(
    tokenizer, 
    daberta_tokenizer, 
    question_sentences,
    save_path,
    vocab_size=20_000,
    min_frequency=100,
    limit_alphabet=200,
    show_progress=False,
)

diff size 10953


In [ ]:
tokenizer = SentencePieceBPETokenizer(vocab=vocab)
tokenizer.pre_tokenizer = Whitespace()
tokenizer.train_from_iterator(
    question_sentences[:200_000],
    vocab_size=5_000,
    min_frequency=10,
    limit_alphabet=200,
    show_progress=False,
)

# tokenizer.save("drive/Shared drives/685 Final Project/Code Search Pre-Trained Model/tokenizer-wiki.json")


# tokenizer.train(
#     question_list,
#     vocab_size=500,
#     # min_frequency=100,
#     # limit_alphabet=200,
#     show_progress=False,
# )

In [ ]:
new_vocab = tokenizer.get_vocab()
diff = {k:v for k,v in new_vocab.items() if k not in vocab}

In [ ]:
len(new_vocab)

5000

In [ ]:
diff